# Exercise Fourteen: Project Design Starter
In this exercise, you'll be planning out a complex project. You'll draw in some code, but focus on commenting to describe your project structure. The sample document below will guide you through organizing and annotating your project design. The primary components you'll include are:

- Dependencies: What modules will your project need?
- Collection: Where is your data coming from?
- Processing: How will you format and process your data?
- Analysis: What techniques will you use to understand your data?
- Visualization: How will you visualize and explore your data?

# Project Overview
I plan on extracting information from Twitter using #BlerDCon. BlerDcon is a convention — the event’s name signals that it specifically caters to blerds (black nerds), BlerDCon is an inclusive space for all POC, LGBTQ+, female, and disabled fans. Pulling data from the 2021 convention, I seek to gather more information and insight into the topics dicussed around this venue and the sentiments of the participants. 

At first, I thought I could pull all the tweets and analyze keywords but ran into some issues, tried to create a HTML file and pull words from there but formatting code causes to many issues. 

Trued editing our location paramaters to make data set larger but data set remains small. 

# Dependencies
Importing modules to extract data from hastags on twitter, compile that information into a digestible list, and then use that info to create visualizations for the data.

In [138]:
# Importing Config to pull credentials for Twitter API
import configparser

#Importing Twitter API
import tweepy

#To turn data into a CSV file
import csv

#Importing to help extract data
import requests
from bs4 import BeautifulSoup

#Importing to map out geographical locations of responses
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#importing to plot findings in visualization
import matplotlib.pyplot as plt

#importing for map visualization
import cartopy.crs as ccrs
from matplotlib.patches import Circle

#importing to assist in compliling 
import re

#Importing for punctuation
from nltk.tokenize import WordPunctTokenizer

#importing to make a wordcloud for visualizations
from wordcloud import WordCloud, STOPWORDS

#To turn work cloud into an image
import numpy as np
from PIL import Image
import random

#Importing to gather sentiments from comments
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

# Importing Pandas to handle collected Twitter data 
import pandas as pd

#trying a new scraper since Tweepy only pulls recent tweets
import snscrape.modules.twitter as sntwitter



ModuleNotFoundError: No module named 'snscrape.modules'

In [ ]:
import snscrape.modules.twitter as sntwitter

In [ ]:
'''import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests'''

# Collection
Collecting data from recent 2021 convention hastag #BlerDcon. Starting date from pull will begin 01-01-2021 to current time. This can be changed later to look into other year conventions. 

Tried to change to #BlerDCon to widen paramaters. 

I checked other avenues of collecting "customer feedback" but Twitter seems to contain the most.

In [ ]:
#Pulling credentials file, so Twiiter Developer account can be accessed
CONFIG = configparser.ConfigParser()
CONFIG.read('credentials.ini')

#Tweepy
auth = tweepy.OAuthHandler(CONFIG['DEFAULT']['consumer_key'], CONFIG['DEFAULT']['consumer_secret'])
auth.set_access_token(CONFIG['DEFAULT']['access_token'], CONFIG['DEFAULT']['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    #dont need to see everything
    print(tweet.text [0:10])

In [ ]:
#pulling info in from specific hashtag
#Need to figure out how to accurately pull all the tweets with specific hasgtag and compile it into a CSV file for analysis.
#Started with BlerDCon2021 and then just BlerdCon
search_words = "#BlerDCon"
date_since = "2010-01-01"
tweets = tweepy.Cursor(api.search_tweets,
              q=search_words, lang="en").items(300)

In [ ]:
import snscrape.modules.twitter as sntwitter
import csv

keyword = 'BlerDCon2021'
maxTweets = 30000

#Open/create a file to append data to
csvFile = open('BlerDCon2021.csv', 'a', newline='', encoding='utf8')

#Use csv writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet'])

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' since:2021-06-01 until:2021-12-16').get_items()) :
        if i > maxTweets :
            break
        csvWriter.writerow([tweet.id, tweet.date, tweet.renderedContent])
csvFile.close()

In [ ]:
'''# Trying another way around Twitter. Found Youtube video discussing topic -- will pull comments from there.
import urllib.request, urllib.error, urllib.parse

url = 'https://www.lipstickalley.com/threads/blerdcon-a-white-girl-wins-the-cosplay-event-is-that-ok.4605659/'

response = urllib.request.urlopen(url)
HTML = response.read()

print(HTML[0: 2000])'''

In [ ]:
'''contents = str(HTML)
startLoc = contents.find("comments")
endLoc = contents.find("<div class='answererUserIcon'>")

print(startLoc)
print(endLoc)

contents = contents[startLoc: endLoc]
print(contents[0: 2000])'''

# Processing
After your data has been collected or imported, store it in a format that works for your purposes. This can vary: for Twitter analysis, it might be a Pandas dataframe, while for text, you might build a document term matrix.

I seem to be successful at pulling some locations but not all tweets. I know the dataset is large cause i've checked it in Twitter. Tried a few exmaples from online but could not get it to work.

In [ ]:
#using pandas to sort tweets
#pulled location out of script to see if username and tweet could just be gathered - this did not worl
tweets_sorted = [[tweet.user.screen_name,tweet.text] for tweet in tweets]

#caluculating coordinates of Tweets -- doesnt look like too many locations (may not use for data set)
#tried removing location to just get tweets
tdf = pd.DataFrame(data=tweets_sorted, columns=['user', 'tweet'])
print(tdf)

#may need to look for word count instead

In [ ]:
# I can create the file but not add the contents of my hastag scrape to it.
# Ideally a CSV will exist with username, location, and tweet.
csvFile = open('BlerDCon', 'a')
csvWriter = csv.writer(csvFile)

In [ ]:
'''tdf = pd.DataFrame(output)
tdf.to_csv('output.csv')'''

In [ ]:
'''#need to organize CSV better then this function will work better

tdf = pd.read_csv('output.csv')
# Prints how pd will structure data types
print(df.dtypes)'''

In [ ]:
'''#trying to read CSV another way
f = open('output.csv', 'r')
text = f.read()
f.close()

#split_string
word_bag = text.split()
print(word_bag[0:100])

#make lower case
text = text.lower()
word_bag = text.split()
print(word_bag[0:100])'''

# Analysis
Would like to analyze the CSV file for common key terms and number of times a topic is mentioned. From their try to build out sentiments of users. 

Seemed to only work with location. Research should focus on common topics and sentiments around the convention -- would have led to controversy from 2021 convention where a white woman won best cosplay at a black convention.

In [ ]:
#this didnt matter cause there was only one location
locs = tdf['location'].value_counts()
print(locs)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='twitter-analysis-client')
limited = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def find_location(row):
    place = row['location']
    location = limited(place)
    
    if location != None:
        return location.latitude, location.longitude
    else:
        return "Not Found", "Not Found"

tdf[['latitude','longitude']] = tdf.apply(find_location, axis="columns", result_type="expand")
tdf

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (20, 10)

# Visualization
The goal was to create a word cloud with common topics from the hashtag and place it over the logo for the conference. A Bokeh chart would be helpful here as well

In [ ]:
counter = tdf.groupby(['latitude','longitude']).size().reset_index().rename(columns={0:'count'})

print(counter)

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
# plot individual locations                                                                                                       
for i in range(0, len(lats)):
    ax.plot(longs[i], lats[i], 'ro', transform=ccrs.PlateCarree())
                           
# add coastlines for reference                                                                                                
ax.coastlines(resolution='50m')
ax.set_global()
def get_radius(freq):
    if freq < 5:
        return 1
    elif freq < 10:
        return 2
    elif freq >= 10:
        return 3
# plot count of tweets per location

colLats = counter['latitude']
colLongs = counter['longitude']
colCounts = counter['count']

for i in range(0, len(counter)):
    ax.add_patch(Circle(xy=[colLongs[i], colLats[i]], radius=get_radius(colCounts[i]), color='blue',alpha=0.5, transform=ccrs.PlateCarree()))
